- [Inverse Projection Transformation](https://towardsdatascience.com/inverse-projection-transformation-c866ccedef1c)

In [2]:
import numpy as np
import cv2
    
# mapping 3d points on to 2d image plane
def projectPoints(X, K, R, t, Kd):
    """ Projects points X (3xN) using camera intrinsics K (3x3),
    extrinsics (R,t) and distortion parameters Kd=[k1,k2,p1,p2,k3].
    
    Roughly, x = K*(R*X + t) + distortion
    
    See http://docs.opencv.org/2.4/doc/tutorials/calib3d/camera_calibration/camera_calibration.html
    or cv2.projectPoints
    """
    
    x = np.asarray(R*X + t)
    
    x[0:2,:] = x[0:2,:]/x[2,:]
    
    r = x[0,:]*x[0,:] + x[1,:]*x[1,:]
    
    x[0,:] = x[0,:]*(1 + Kd[0]*r + Kd[1]*r*r + Kd[4]*r*r*r) + 2*Kd[2]*x[0,:]*x[1,:] + Kd[3]*(r + 2*x[0,:]*x[0,:])
    x[1,:] = x[1,:]*(1 + Kd[0]*r + Kd[1]*r*r + Kd[4]*r*r*r) + 2*Kd[3]*x[0,:]*x[1,:] + Kd[2]*(r + 2*x[1,:]*x[1,:])

    x[0,:] = K[0,0]*x[0,:] + K[0,1]*x[1,:] + K[0,2]
    x[1,:] = K[1,0]*x[0,:] + K[1,1]*x[1,:] + K[1,2]
    
    return x

In [15]:
src = np.ones((6, 3))
src[:,1] = 2
src[:,2] = range(6) # source points
rvec = np.array([0,0,0], np.float) # rotation vector
tvec = np.array([0,0,0], np.float) # translation vector
fx = fy = 1 # meters
cx = cy = 0.0
cameraMatrix = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]])
result = cv2.projectPoints(src, rvec, tvec, cameraMatrix, None)

In [10]:
for n in range(len(src)):
    print(src[n], '==>', result[0][n])

[1. 2. 0.] ==> [[0.03 0.06]]
[1. 2. 1.] ==> [[0.03 0.06]]
[1. 2. 2.] ==> [[0.015 0.03 ]]
[1. 2. 3.] ==> [[0.01 0.02]]
[1. 2. 4.] ==> [[0.0075 0.015 ]]
[1. 2. 5.] ==> [[0.006 0.012]]


In [5]:
src

array([[1., 2., 0.],
       [1., 2., 1.],
       [1., 2., 2.],
       [1., 2., 3.],
       [1., 2., 4.],
       [1., 2., 5.]])

In [6]:
cameraMatrix

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])